In [1]:
# define a model using pytorch
# a pretrained model adding layers to classify the 4 classes of the images
import torch.nn as nn
from torchvision import models
from torchvision.models.resnet import ResNet18_Weights

class MIDmodel(nn.Module):
    def __init__(self):
        super(MIDmodel, self).__init__()
        # first pretrained
        self.pretrained = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # add layer(s)
        self.fc1 = nn.Linear(1000, 4)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.pretrained(x)
        # x = self.dropout(x)
        x = self.fc1(x)
        return x


c:\Users\Davide\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
class ImageDataset(Dataset):
    def __init__(self, root_dir, patients_ids, patients_df, transform=None):
        # select the rows of the dataframe that correspond to the patients in the list
        self.patients_df = patients_df[patients_df["patient_id"].isin(patients_ids)]
        self.root_dir = root_dir
        self.idxs = set()
        if transform:
            self.transform = transform
            # duplicate the patients_df, so the network will see each image twice, one standard and one transformed
            self.patients_df = pd.concat([self.patients_df] * 2, ignore_index=True)
        else:
            self.transform = None
               
            
    def __len__(self):
        return len(self.patients_df)

    def __getitem__(self, idx):
        # get idx row of the dataframe
        row = self.patients_df.iloc[idx]
        # path = root_dir/patientid_examid_spot_framenumber_score.png
        path = os.path.join(self.root_dir, row["patient_id"] + "_" + row["exam_id"] + "_" + row["spot"] + "_" + row["frame_number"] + "_" + row["score"] + ".png")
        image = Image.open(path)
        label = row["score"]
        # if the image is grayscale, convert it to RGB (used to test with the smaller dataset)
        # if image.mode != "RGB":
        #     image = image.convert("RGB")
               
        if self.transform:
            if path not in self.idxs: # if it's the first time we see the image, add its path to the set
                self.idxs.add(path)
            else: # if it's the second time we see the image, apply the transform and remove its path from the set 
                image = self.transform(image)
                self.idxs.remove(path)
        # in any case, transform to tensor
        image = transforms.ToTensor()(image)
        image = transforms.Resize((224, 224))(image)
        # Finally the values are first rescaled to [0.0, 1.0] and then normalized using mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
        image = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(image)
      
        return image, int(label)

In [3]:
import pandas as pd
import torch
import glob
data_dir = "images/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images_paths = glob.glob(f"{data_dir}*.png", recursive=True)
# images are named as: patientid_examid_spotnumber_framenumber_score.png
# create a dataframe with the data removing "images/"
images_df = pd.DataFrame([path[len(data_dir):-4].split("_") for path in images_paths], columns=["patient_id", "exam_id", "spot", "frame_number", "score"])
images_df["score"] = images_df["score"].astype(str)
images_df["frame_number"] = images_df["frame_number"].astype(str)
images_df["spot"] = images_df["spot"].astype(str)
images_df["patient_id"] = images_df["patient_id"].astype(str)
images_df["exam_id"] = images_df["exam_id"].astype(str)
patients_ids = set(images_df["patient_id"])
patients_ids = list(patients_ids)

# select the 8 patients for training based on the most balanced distribution of the scores
import itertools
combs = list(itertools.combinations(patients_ids, 8))
stds = []
for i, c in enumerate(combs):
    stds.append((images_df[images_df["patient_id"].isin(c)].groupby("score").count()["patient_id"].std(), i))

train_patients = [x for x in combs[min(stds)[1]]]
print(train_patients)
print(images_df[images_df["patient_id"].isin(train_patients)].groupby("score").count()["patient_id"])

test_patients = [x for x in patients_ids if x not in train_patients]
print(test_patients)
print(images_df[images_df["patient_id"].isin(test_patients)].groupby("score").count()["patient_id"])

['1017', '1069', '1048', '1047', '1050', '1051', '1052', '1067']
score
0    7888
1    7540
2    7189
3    7592
Name: patient_id, dtype: int64
['1066', '1068', '1045']
score
0    6625
1    1721
2    5535
3    3631
Name: patient_id, dtype: int64


In [4]:
# transformations from the paper Deep Learning for Classification and Localization of COVID-19 Markers in Point-of-Care Lung Ultrasound
# were each activated on the image-label pair with a probability of 0.33. The set of augmentation functions, each applied with a randomly sampled strength bounded by a set maximum, consists of:
# - affine transformations (translation (max. ±15%), rotation (max. ±15◦), scaling (max. ±45%), and shearing (max. ±4.5◦))
# - multiplication with a constant (max. ±45%)
# - Gaussian blurring (σmax = 3/4 )
# - contrast distortion (max. ±45%)
# - horizontal flipping (p = 0.5)
# - additive white Gaussian noise (σmax = 0.015)
transformations = transforms.Compose([
    transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.15, 0.15), scale=(0.55, 1.45), shear=4.5)], p=0.5),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.45, contrast=0.45, saturation=0.45, hue=0.45)], p=0.5),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3, sigma=(0.75))], p=0.5),
    transforms.RandomApply([transforms.RandomHorizontalFlip(p=0.5)], p=0.5)
    ])

In [5]:
from torch import optim
from tqdm import tqdm

def train_model(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects/ len(dataloader.dataset)
    return epoch_loss, epoch_acc

def test_model(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects / len(dataloader.dataset)
    return epoch_loss, epoch_acc


In [6]:
data_dir = "images/"
num_epochs = 100
learning_rate = 0.0001

model = MIDmodel()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = ImageDataset(data_dir, train_patients, images_df, transformations)
test_dataset = ImageDataset(data_dir, test_patients, images_df)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
results_train = []
results_test = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    train_loss, train_acc = train_model(model, train_dataloader, optimizer, criterion, device)
    results_train.append([train_loss, train_acc])
    print(f"Training Loss: {train_loss:.4f} Training Accuracy: {train_acc:.4f}")
    test_loss, test_acc = test_model(model, test_dataloader, criterion, device)
    results_test.append([test_loss, test_acc])
    print(f"Test Loss: {test_loss:.4f} Test Accuracy: {test_acc:.4f}")
    if epoch % 5 == 1:
        torch.save(model.state_dict(), f"MIDmodel_{epoch}.pt")

print("Train: ", results_train)
print("Test: ", results_test)

# save the model
torch.save(model.state_dict(), f"MIDmodel.pt")

In [7]:
# load the last model
model = MIDmodel()
model.load_state_dict(torch.load("MIDmodel_1.pt"))
model.to(device)
model.eval()

# iterate over test dataset and save softmax outputs for the wrong predictions
wrong_predictions = []
for i in tqdm(range(len(test_dataset))):
    image, label = test_dataset[i]
    image = image.unsqueeze(0)
    image = image.to(device)
    output = model(image)
    _, pred = torch.max(output, 1)
    if pred != label:
        # TODO: save result in a file since low GPU memory
        # wrong_predictions.append((i, output))
        pass


  9%|▊         | 1511/17512 [00:24<04:22, 60.91it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 12.00 GiB total capacity; 10.83 GiB already allocated; 0 bytes free; 11.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF